In [1]:
import pandas as pd
import json
import dateparser
from datetime import datetime
import math
import re
from collections import Counter
from tqdm import tqdm
import string
import pickle
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix

# Data Processing

### Data Loading

In [2]:
class CandidateRecord():
    def __init__(self, educations, experiences, seniority):
        self.educations = educations
        self.experiences = experiences
        self.seniority = seniority

class EducationRecord():
    def __init__(self, school, description, degree, time):
        self.school = school
        self.description = description
        self.degree = degree
        self.time = time

class ExperienceRecord():
    def __init__(self, work, description, title, time, skills):
        self.employer = work
        self.description = description
        self.title = title
        self.skills = skills
        self.time = time


In [3]:
train_filepath = "./data/seniority.train"
test_filepath = "./data/seniority.test"

def load_data(filepath):
    candidates = []
    with open(filepath, "r") as infile:
        for line in infile:
            record = json.loads(line.strip())
            educations = [EducationRecord(e['school'], e['description'], e['degree'], e['time']) for e in record['education']]
            experiences =[ExperienceRecord(e['work'], e['description'], e['title'], e['time'], e['skills']) for e in record['experience']]
            candidate = CandidateRecord(educations, experiences, record['seniority_level'])
            candidates.append(candidate)

    return candidates

train_candidates = load_data(train_filepath)
test_candidates = load_data(test_filepath)



### Data Exploration

In [4]:
len(train_candidates), len(test_candidates)

(29969, 29999)

In [5]:
train_sen = [c.seniority for c in train_candidates]
test_sen = [c.seniority for c in test_candidates]

In [6]:
Counter(train_sen), Counter(test_sen)

(Counter({'Entry': 9927,
          'Senior': 3564,
          'Mid-Level': 8947,
          'Intern': 928,
          'Manager': 3892,
          'Director': 1768,
          None: 294,
          'Vice President': 447,
          'CXO': 202}),
 Counter({'Mid-Level': 9090,
          'Manager': 4073,
          'Senior': 4196,
          'Director': 2135,
          'Entry': 8362,
          'Vice President': 674,
          None: 272,
          'CXO': 303,
          'Intern': 894}))

### Data Cleaning

#### Time data

Time data seems to have various formats and arbitrary values. We will have to clean these to extract duration of work experience or figure out if the candidate has completed their degree.

The following code was written iteratively:
- First I eye-balled the time data for various samples
- Then I looked at text-only time data
- Then I looked at unknown time data (empty or "n/a" or "unknown")
- Then I added education/experience count to check if total count makes sense
    - This revealed that for some education/experience records, we have a single time



In [7]:
error_times = set()
textual_times = set()
textual_times_count = 0
empty_times_count = 0
total_times_count = 0
education_count = 0
for c in train_candidates:
    for e in c.educations:
        education_count += 1
        if e.time == []:
            empty_times_count += 2
        for time in e.time:
            total_times_count += 1
            try:
                if bool(re.match(r'^\D*$', time)):
                    textual_times.add(time)
                    textual_times_count += 1
            except Exception as e:
                error_times.add(time)

textual_times, textual_times_count, empty_times_count, total_times_count, education_count

({'', 'notKnown'}, 17054, 22866, 72403, 48242)

In [8]:
error_times = set()
textual_times = set()
textual_times_count = 0
empty_times_count = 0
total_times_count = 0
unknown_times_count = 0
experience_count = 0
for c in train_candidates:
    for e in c.experiences:
        experience_count += 1
        if e.time == []:
            empty_times_count += 2
            unknown_times_count += 2
        for time in e.time:
            total_times_count += 1
            try:
                if time in ['', 'N/A', 'notKnown']:
                    unknown_times_count += 1
                if bool(re.match(r'^\D*$', time)):
                    textual_times.add(time)
                    textual_times_count += 1
            except Exception as e:
                error_times.add(time)

textual_times, unknown_times_count, textual_times_count, empty_times_count, total_times_count, experience_count

({'',
  'Aujourd’hui',
  'Current',
  'N/A',
  'Present',
  'Presente',
  'Till Date',
  'current',
  'notKnown',
  'present'},
 989,
 27095,
 28,
 293478,
 152830)

So, for ~12% (0.5 * 23k/95k) of the education data and for ~0.16% (0.5 * 1k/300k) of the experience data, we do not have a valid time.

Assumptions:
1. I think we can ignore the time of graduation altogether since we do not have plan to extract any feature based on it. If we do want to check if, say, the person has completed their degree, then we would need a valid end time.

2. We can treat the 0.16% missing times for experience data as missing data. Can choose to ignore it considering its low volume.

3. I ran dateparser.parse() on all the above times and it ran successfully. Thus, I am assuming we either have valid dates or "text-only" dates that are shown above. If there is any date like "2022abc", then we count it as just another invalid date.

In [9]:
a = dateparser.parse("2022abc")
a == None

True

### College Degree 

In [10]:
degrees = []
for c in train_candidates:
    for e in c.educations:
        degrees.append(e.degree)
    
len(degrees), degrees[:10]

(48242,
 ['Master of Business Administration - MBA',
  'Bachelors of Science in Business Administration with a Concentration in Marketing',
  '',
  'Master of Science in Finance',
  'B.A in Applied Mathematics',
  'GED',
  'M.S.',
  'Bachelor of Science',
  'Bachelor of Arts',
  ''])

Need to remove punctuation to get terms like "be", "bsc", "msc" instead of "b", "e", "sc", "m"

The above list is also cluttered by department names. We can use a standand list of departments at stop words to fix this.

In [11]:
# Source: https://www.mydegreeguide.com/types-of-degrees/types-of-bachelor-degree/
# The dept list given above is augmented with the individual tokens found in the dept names itself e.g. "business administration" => "business", "administration"
depts = ["architecture", "arts", "business", "business administration", "science in business", "canon law", "computer science", "science in computer science", "criminal justice", "science in criminal justice", "divinity", "education", "science in education", "wireless engineering", "engineering", "science in engineering", "science in aerospace engineering", "science in agricultural engineering", "science in biological systems", "science in biosystems and agricultural engineering", "science in biological engineering", "biomedical engineering", "science in biomedical engineering", "science in chemical engineering", "science in chemical and biomolecular engineering", "science in chemical and materials engineering", "civil engineering", "science in civil engineering", "science in civil and infrastructure engineering", "computer engineering", "science in computer engineering", "science in computer science and engineering", "science in electrical and computer engineering", "electrical engineering", "science in electrical engineering", "science in engineering management", "science in environmental engineering", "fiber engineering", "science in industrial engineering", "science in manufacturing engineering", "science in manufacturing systems engineering", "science in materials science and engineering", "science in materials engineering", "mechanical engineering", "science in mechanical engineering", "science in metallurgical engineering", "science in mining engineering", "science in systems", "software engineering", "science in software engineering", "systems engineering", "science in systems engineering", "engineering technology", "science in engineering technology", "science in civil engineering technology", "science in computer engineering technology", "science in construction engineering technology", "science in drafting design technology", "science in electrical/electronics technology", "science in electrical engineering technology", "science in electro-mechanical engineering technology", "science in mechanical engineering technology", "fine arts", "forestry", "science in forest research", "hebrew letters", "journalism", "laws", "liberal studies", "literature", "marine science", "music", "nursing", "science in nursing", "pharmacy", "philosophy", "religious education", "science", "science in chemistry", "technology", 'administration', 'aerospace', 'agricultural', 'and', 'biological', 'biomedical', 'biomolecular', 'biosystems', 'canon', 'chemical', 'chemistry', 'civil', 'computer', 'construction', 'criminal', 'design', 'drafting', 'electrical', 'electro', 'electronics', 'environmental', 'fiber', 'fine', 'forest', 'hebrew', 'in', 'industrial', 'infrastructure', 'justice', 'law', 'letters', 'liberal', 'management', 'manufacturing', 'marine', 'materials', 'mechanical', 'metallurgical', 'mining', 'religious', 'research', 'software', 'studies', 'systems', 'wireless']

In [12]:
degrees = [s.translate(str.maketrans('', '', string.punctuation)) for s in degrees]

degree_vectorizer = CountVectorizer(stop_words=depts + ["of"], ngram_range=(1, 2), max_features=50)
cv_fit = degree_vectorizer.fit_transform(degrees)
popular_degrees = degree_vectorizer.get_feature_names_out()
popular_degree_freq = cv_fit.toarray().sum(axis=0)

sorted(zip(popular_degree_freq, popular_degrees), reverse=True)

[(10343, 'bachelor'),
 (4202, 'master'),
 (3740, 'degree'),
 (2705, 'bs'),
 (2643, 'diploma'),
 (2572, 'bachelors'),
 (1947, 'mba'),
 (1889, 'masters'),
 (1653, 'ba'),
 (1434, 'certificate'),
 (1403, 'ms'),
 (1370, 'school'),
 (1265, 'high'),
 (1235, 'high school'),
 (1162, 'bachelors degree'),
 (1085, 'information'),
 (1016, 'school diploma'),
 (866, 'associate'),
 (708, 'bachelor degree'),
 (707, 'associates'),
 (690, 'masters degree'),
 (648, 'certification'),
 (629, 'marketing'),
 (565, 'be'),
 (546, 'economics'),
 (514, 'applied'),
 (509, 'bachelor bs'),
 (477, 'phd'),
 (477, 'bsc'),
 (473, 'finance'),
 (418, 'general'),
 (396, 'master ms'),
 (394, 'international'),
 (386, 'commerce'),
 (376, 'btech'),
 (363, 'communication'),
 (346, 'mathematics'),
 (345, 'accounting'),
 (336, 'communications'),
 (301, 'ged'),
 (300, 'master degree'),
 (294, 'associates degree'),
 (286, 'bba'),
 (277, 'certified'),
 (277, 'bachelor commerce'),
 (274, 'psychology'),
 (271, 'msc'),
 (269, 'doctor')


This looks good!

In [13]:
sum(degree_vectorizer.transform(["bs", "ms"])).A

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0]], dtype=int64)


For titles, we don't need to remove any punctuation or other terms. We can simply count the n-grams.

In [14]:
titles = []
for c in train_candidates:
    for e in c.experiences:
        titles.append(e.title)
    
len(titles), titles[:50]

(152830,
 ['General Laborer',
  'Produce Associate',
  'Dietary Aide',
  'Sr. Supply Planner',
  'Supply Chain Manager',
  'Master Scheduler',
  'Master Scheduler',
  'Master Scheduler',
  'Master Scheduler',
  'Material Resource Planner',
  'Utility Worker',
  'Ramp Agent',
  'Lead',
  'Roasting lead',
  'Second Night Lead',
  '',
  'Fixed Income & Absolute Return Team Intern',
  'Real Estate Team Intern',
  'Financial Service Department Intern',
  'Shipping Clerk',
  'Unemployed',
  'Lift Driver/Module Operator',
  'Unemployed',
  'Order Picker/Unloader',
  'Global Environmental Expert',
  'Global Environmental Program Manager',
  'Global Environmental Program Manager',
  'Senior Health, Safety & Environment Manager',
  'Environmental Specialist, Latin America',
  'Manager, Safety & Environment',
  'Senior Pricing Manager',
  'Sr. Pricing & Wireless Solution Manager',
  'International Market Systems Engineer',
  '',
  'Engineering Section Manager',
  'Senior Systems Engineer',
  'Sen

In [15]:
title_vectorizer = CountVectorizer(stop_words='english', ngram_range=(1, 3), max_features=100)
cv_fit = title_vectorizer.fit_transform(titles)
popular_titles = title_vectorizer.get_feature_names_out()
popular_title_freq = cv_fit.toarray().sum(axis=0)

sorted(zip(popular_title_freq, popular_titles), reverse=True)

[(28073, 'manager'),
 (13544, 'engineer'),
 (11089, 'senior'),
 (9008, 'sales'),
 (8814, 'analyst'),
 (8733, 'software'),
 (8295, 'assistant'),
 (8027, 'intern'),
 (7810, 'director'),
 (6626, 'consultant'),
 (6365, 'business'),
 (5754, 'software engineer'),
 (5445, 'marketing'),
 (5143, 'project'),
 (4796, 'account'),
 (4630, 'associate'),
 (4569, 'development'),
 (4518, 'lead'),
 (4214, 'executive'),
 (4020, 'specialist'),
 (3583, 'operations'),
 (3425, 'developer'),
 (3356, 'sr'),
 (3039, 'coordinator'),
 (3038, 'research'),
 (3036, 'project manager'),
 (2894, 'management'),
 (2880, 'technical'),
 (2799, 'product'),
 (2697, 'customer'),
 (2599, 'services'),
 (2596, 'service'),
 (2574, 'supervisor'),
 (2494, 'data'),
 (2371, 'engineering'),
 (2329, 'account manager'),
 (2325, 'team'),
 (2292, 'support'),
 (2261, 'representative'),
 (2228, 'president'),
 (2111, 'program'),
 (2023, 'systems'),
 (1965, 'financial'),
 (1844, 'technician'),
 (1831, 'operator'),
 (1786, 'business developmen

This list looks decent. Most of the terms give an indication of the seniority of the corresponding role. I tried `max_features = 200` and saw not so informative terms at the end, so switched to 100 and we can see that the last few terms also have good indicative information.

In [16]:
sum(title_vectorizer.transform(["software engineer", "machine learning engineer"])).A

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [17]:
skills = []
for c in train_candidates:
    for e in c.experiences:
        if e.skills:
            skills += e.skills
    
len(skills), skills[:50]

(1361721,
 ['production',
  'labor',
  'floors',
  'safety',
  'damages',
  'customer service',
  'cleaning',
  'truck',
  'holidays',
  'fruit',
  'associate',
  'display',
  'production',
  'breakfast',
  'lunch',
  'customer service',
  'supply chain',
  'replenishment',
  'team',
  'operational metrics',
  'wip',
  'metrics',
  'components',
  'customer service',
  'supply chain',
  'replenishment',
  'team',
  'operational metrics',
  'metrics',
  'components',
  'wip',
  'range planning',
  'manufacturing',
  'projects',
  'planning',
  'material planning',
  'process',
  'procurement',
  'customer service',
  'manufacturing',
  'diverse team',
  'project',
  'inventory accuracy',
  'delivery',
  'price',
  'purchasing',
  'buyer',
  'vendors',
  'planning',
  'inventory'])

In [18]:
skills_vectorizer_unigram = CountVectorizer(max_features=200)
cv_fit = skills_vectorizer_unigram.fit_transform(skills)
popular_skills = skills_vectorizer_unigram.get_feature_names_out()
popular_skill_freq = cv_fit.toarray().sum(axis=0)

sorted(zip(popular_skill_freq, popular_skills), reverse=True)

[(51347, 'management'),
 (44808, 'sales'),
 (24902, 'business'),
 (23901, 'marketing'),
 (19089, 'data'),
 (17660, 'process'),
 (17017, 'analysis'),
 (15267, 'development'),
 (14516, 'customer'),
 (14225, 'operations'),
 (13284, 'training'),
 (11384, 'strategy'),
 (10694, 'financial'),
 (10072, 'service'),
 (10001, 'quality'),
 (9407, 'product'),
 (9321, 'planning'),
 (9285, 'clients'),
 (9098, 'software'),
 (8656, 'reports'),
 (8403, 'inventory'),
 (8287, 'solutions'),
 (8250, 'projects'),
 (8218, 'team'),
 (8144, 'project'),
 (7804, 'design'),
 (7773, 'production'),
 (7720, 'leadership'),
 (7555, 'implementation'),
 (7345, 'revenue'),
 (7197, 'research'),
 (6828, 'maintenance'),
 (6720, 'budget'),
 (6552, 'web'),
 (6545, 'support'),
 (6355, 'engineering'),
 (6355, 'cost'),
 (6320, 'application'),
 (6313, 'services'),
 (6046, 'employees'),
 (6018, 'integration'),
 (5894, 'media'),
 (5736, 'safety'),
 (5702, 'database'),
 (5544, 'office'),
 (5361, 'delivery'),
 (5348, 'tools'),
 (5311,

If we looked at skills n-grams in range (1, 3), we notice that they are dominated by unigrams. Thus, I have decided to take 200 unigrams and 100 bi/tri-grams (most of which are bigrams).

In [19]:
skills_vectorizer_bitri = CountVectorizer(max_features=100, ngram_range=(2, 3))
cv_fit = skills_vectorizer_bitri.fit_transform(skills)
popular_skills = skills_vectorizer_bitri.get_feature_names_out()
popular_skill_freq = cv_fit.toarray().sum(axis=0)

sorted(zip(popular_skill_freq, popular_skills), reverse=True)

[(4944, 'customer service'),
 (2753, 'social media'),
 (2602, 'business development'),
 (2275, 'supply chain'),
 (2120, 'cross functional'),
 (1738, 'project management'),
 (1723, 'sales team'),
 (1578, 'product development'),
 (1390, 'customer satisfaction'),
 (1273, 'human resources'),
 (1245, 'business requirements'),
 (1239, 'quality assurance'),
 (1204, 'management system'),
 (1193, 'data analysis'),
 (1183, 'process improvement'),
 (1141, 'software development'),
 (1126, 'business process'),
 (1087, 'change management'),
 (1061, 'continuous improvement'),
 (1006, 'account management'),
 (992, 'technical support'),
 (960, 'quality control'),
 (935, 'market research'),
 (880, 'business intelligence'),
 (877, 'performance management'),
 (851, 'risk management'),
 (818, 'business units'),
 (790, 'in house'),
 (784, 'professional services'),
 (782, 'product management'),
 (746, 'employee relations'),
 (742, 'senior management'),
 (736, 'market share'),
 (716, 'strategic planning'),
 (

In [20]:
sum(skills_vectorizer_unigram.transform(["technology", "engineer", "communication"])).A[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0], dtype=int64)

We can use the tokenizer from Spacy if needed. Note we are using the English tokenizer here but it seems to do a decent job on French sentences too.

I tried using this tokenizer instead of the default tokenizer in CountVectorizer but the combination was not playing well around punctuations. And the default tokenizer does a decent job anyway, so I decided to keep it as is.

In [21]:
from spacy.lang.en import English
nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
tokenizer = nlp.tokenizer

doc = tokenizer("inventory.\n\u25cf Unloaded freights/pallets from trailers.\n\u25cf Stocked and organized shelves according to procedures and policies.\n\u25cf Staged promotional")
frenchdoc = tokenizer("\u2022 V\u00e9rifier les ingr\u00e9dients d'adh\u00e9sif pr\u00e9par\u00e9s par le journalier (deuxi\u00e8me compte des ingr\u00e9dients).\n\u2022 Ins\u00e9rer les ingr\u00e9dients dans le m\u00e9langeur selon la proc\u00e9dure \u00e9tablie, afin d'obtenir un produit de premi\u00e8re qualit\u00e9 et ainsi rencontrer les normes de tests \u00e9tablies.\n\u2022 Proc\u00e9der au set-up de la ligne d'extrusion, c'est-\u00e0-dire couteau, rouleau, ")


In [22]:
doc, len(doc), doc[0], type(doc[0])

(inventory.
 ● Unloaded freights/pallets from trailers.
 ● Stocked and organized shelves according to procedures and policies.
 ● Staged promotional,
 27,
 inventory,
 spacy.tokens.token.Token)

In [23]:
frenchdoc, len(frenchdoc), frenchdoc[1]

(• Vérifier les ingrédients d'adhésif préparés par le journalier (deuxième compte des ingrédients).
 • Insérer les ingrédients dans le mélangeur selon la procédure établie, afin d'obtenir un produit de première qualité et ainsi rencontrer les normes de tests établies.
 • Procéder au set-up de la ligne d'extrusion, c'est-à-dire couteau, rouleau, ,
 66,
 Vérifier)

### Test out DistilBert's embeddings

In [24]:

text_to_embed = """
This is a really long piece of text... The AI / Machine Learning team is building the industry's leading Machine Learning models for use cases at scale. This team is also responsible for pushing the boundaries of applied Machine Learning on a challenging and diverse dataset.  
What you will do (learn to do) 
You will own, train, build and deploy cutting edge models across all Eightfold products, end to end- in production 
Create innovative algorithms for Machine Learning & AI 
Create industry best practices for Machine Learning in Recruiting and HR 
Create industry best practices for Machine Learning in Recruiting and HR 
Create industry best practices for Machine Learning in Recruiting and HR 
Do it responsibly to provide equal opportunity for everyone by extending our internal model fairness platform 
Create innovative algorithms for Machine Learning & AI 
Analyze data to identify actionable insights and optimize deployment results 
Develop AI-based systems for Natural Language Processing (NLP) 
Implement best practices for building AI-enabled products 
Utilize Python programming language to web scrape and develop databases 
Optimize Machine Learning models for time efficiency, performance, cost, scalability, and accuracy 
Utilize state-of-the-art techniques such as AutoML and transfer learning 
Develop tools and processes for automatically extracting information from text using Bert 
Do it responsibly to provide equal opportunity for everyone by extending our internal model fairness platform 
Create innovative algorithms for Machine Learning & AI 
Analyze data to identify actionable insights and optimize deployment results 
Develop AI-based systems for Natural Language Processing (NLP) 
Implement best practices for building AI-enabled products 
Utilize Python programming language to web scrape and develop databases 
Optimize Machine Learning models for time efficiency, performance, cost, scalability, and accuracy 
Utilize state-of-the-art techniques such as AutoML and transfer learning 
Develop tools and processes for automatically extracting information from text using Bert 
Do it responsibly to provide equal opportunity for everyone by extending our internal model fairness platform 
Create innovative algorithms for Machine Learning & AI 
Analyze data to identify actionable insights and optimize deployment results 
Develop AI-based systems for Natural Language Processing (NLP) 
Implement best practices for building AI-enabled products 
Utilize Python programming language to web scrape and develop databases 
Optimize Machine Learning models for time efficiency, performance, cost, scalability, and accuracy 
Utilize state-of-the-art techniques such as AutoML and transfer learning 
Develop tools and processes for automatically extracting information from text using Bert 
What you bring 
Good foundation in Machine Learning (ML), Deep Learning, and NLP 
Hands-on experience in applying Natural Language Processing (NLP) solutions is a plus. 
Ability to work cross functionally & interface with data science experts 
Familiar with Language models, transformers like BERT, GPT-3, T-5 etc. 
Strong knowledge of CS fundamental concepts and ML languages ( like Python, C, C++, Java, JavaScript, R, and Scala, etc. ) 
Ability to innovate, as proven by a track record of software artifacts or academic publications in applied machine learning. 
Understanding of data and/or ML systems with ability to think across layers of the stack - REST APIs, microservices, data ingestion and processing systems, and distributed systems. 
Nice To Haves 
Experience with scientific libraries in Python (numba, pandas) and machine learning tools and frameworks (scikit-learn, tensorflow, torch, etc.). 
Experience of implementing production machine learning systems, working with large scale datasets is a plus 
Solid understanding of machine learning theory 
Metrics-focused and passionate about delivering high quality models. 
Familiar with Pandas or Python machine learning libraries 
Familiar with Spark, MLLib, Databricks MLFlow, Apache Airflow and similar related technologies. 
Familiar with a cloud based environment such as AWS 
Experience with analyzing large data sets, using Hadoop, Spark or related technologies is a plus. 
CA Pay Transparency 
The information below is provided for candidates hired in California location. 
In California, the standard base pay range for this role is USD $121,000 - $163,000 annually. 
In addition to a competitive base salary, this position is also eligible for equity awards, benefits and discretionary bonuses. A candidate’s salary is determined by various factors including, but not limited to, experience, skills, and geographic location within the state. 
The information below is provided for candidates hired in California location. 
In California, the standard base pay range for this role is USD $121,000 - $163,000 annually. 
In addition to a competitive base salary, this position is also eligible for equity awards, benefits and discretionary bonuses. A candidate’s salary is determined by various factors including, but not limited to, experience, skills, and geographic location within the state
"""

In [25]:
def break_list_into_chunks(lst, K, pad_item):
    # Calculate the number of chunks required
    num_chunks = (len(lst) + K - 1) // K

    # Initialize an empty list to store the result
    result = []

    # Break the list into chunks of size K
    for i in range(num_chunks):
        start_idx = i * K
        end_idx = (i + 1) * K
        chunk = lst[start_idx:end_idx]

        # Pad the last chunk if its length is less than K
        if len(chunk) < K:
            chunk.extend([pad_item] * (K - len(chunk)))

        result.append(chunk)

    return result

N = 10
K = 3
input_list = [i for i in range(N)]
padding_item = 5

result = break_list_into_chunks(input_list, K, padding_item)
print(result)


[[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 5, 5]]


In [26]:
from transformers.utils.logging import set_verbosity_error
# Prevents the warning that shows up whenever our input text length is greater than model's max length
set_verbosity_error()

In [27]:
from transformers import DistilBertTokenizer, DistilBertModel
from optimum.bettertransformer import BetterTransformer

model = DistilBertModel.from_pretrained('distilbert-base-uncased')
model.to(0)
model = BetterTransformer.transform(model)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [30]:
import torch

def get_distilbert_embeddings_averaged(model, tokenizer, text, chunk_size=512):
    input_ids = tokenizer.encode(text)
    chunks = break_list_into_chunks(input_ids, chunk_size, tokenizer.pad_token_id)
    
    model.eval()
    with torch.no_grad():
        chunk_embeddings = [model(torch.tensor(chunk).unsqueeze(0).to(0))[0] for chunk in chunks]
    
    cat_emb = torch.cat(chunk_embeddings)
    mean_embedding = torch.mean(cat_emb, dim=[0, 1])
    
    return mean_embedding

chunk_size = 512
mean_embedding = get_distilbert_embeddings_averaged(model, tokenizer, text_to_embed, chunk_size)

mean_embedding.shape, mean_embedding

(torch.Size([768]),
 tensor([-2.6178e-01,  2.6635e-01,  3.0354e-01,  1.6429e-01,  3.0573e-01,
          6.6116e-02,  5.0776e-02,  2.2882e-01, -6.8600e-02, -1.8553e-01,
          6.2509e-02, -1.1902e-01, -1.8368e-01,  1.7169e-01,  2.7855e-02,
          3.7834e-01,  2.3089e-01,  1.3869e-01, -1.4346e-01,  4.8342e-01,
          2.7682e-01,  7.8471e-02,  9.8396e-02,  4.3808e-01,  3.1439e-01,
         -9.5011e-02,  1.0128e-02, -1.3841e-01, -1.9912e-01,  3.2196e-02,
          3.2758e-01, -1.4449e-01, -1.4969e-01, -2.7410e-01, -3.5431e-02,
          1.0853e-01, -2.6034e-02, -9.0840e-02, -4.7375e-02,  1.4446e-01,
         -5.2179e-01, -1.3926e-01,  1.1762e-02,  7.1548e-02, -2.0224e-01,
         -2.6292e-01,  1.2055e-01, -6.5668e-02,  7.0362e-02, -5.8428e-02,
         -4.1147e-01,  2.2228e-01, -8.0147e-02, -3.2101e-02,  1.9478e-01,
          3.9656e-01, -1.3091e-01, -4.4888e-01, -5.2075e-01, -3.9475e-01,
          1.1454e-02, -1.9935e-02,  1.3299e-02, -2.6583e-01, -1.3147e-02,
          9.5702e-

In [31]:
a = [10]
a += (mean_embedding.tolist())
a

[10,
 -0.26177769899368286,
 0.2663472294807434,
 0.30354487895965576,
 0.16428805887699127,
 0.30572718381881714,
 0.06611596792936325,
 0.050775691866874695,
 0.22881734371185303,
 -0.06859952211380005,
 -0.18552951514720917,
 0.0625094473361969,
 -0.11901751160621643,
 -0.1836758852005005,
 0.17169272899627686,
 0.027855176478624344,
 0.378335177898407,
 0.23089271783828735,
 0.13868936896324158,
 -0.1434575915336609,
 0.48342272639274597,
 0.276824414730072,
 0.07847149670124054,
 0.09839555621147156,
 0.438082218170166,
 0.31439435482025146,
 -0.09501055628061295,
 0.010127642191946507,
 -0.13841107487678528,
 -0.19911806285381317,
 0.03219575062394142,
 0.32757633924484253,
 -0.14448881149291992,
 -0.14968812465667725,
 -0.2741044759750366,
 -0.03543129563331604,
 0.10852749645709991,
 -0.02603449672460556,
 -0.09084025770425797,
 -0.04737519472837448,
 0.14446407556533813,
 -0.5217850208282471,
 -0.1392594426870346,
 0.01176176406443119,
 0.07154843956232071,
 -0.202240616083145

# Feature Engineering

Here are some features we can consider for a classification task:
1. Years of Work Experience
    - Fairly straightforward numerical feature
    - We will merge overlapping work experiences for simplicity -- can perhaps do a weighted sum in some fashion
2. Education level
    - We can have a feature corresponding to an education at each level (certificate, diploma, associate, bachelor, master, phd)
    - We can also use a CounterVectorizer -- that should capture the above unigram terms
3. Job Title is a strong indicator
    - Look for the presence of certain terms, perhaps perform one-hot encoding on the n-grams
4. Skills
    - Can create feature vectors from skills -- same as above, we can start with a CountVectorizer for the n-grams
    - Skills are specific to the industry/field but can be a good indicator of seniority.
5. Work and education descriptions
    - Can chunk the descriptions and create averaged LLM embeddings of the descriptions
6. Company and College ranking based features
    - If we have some ranking of various companies, we can perhaps create a feature out of this
    - Manager or CEO at Google probably denotes higher seniority than counterparts at other companies


*Let's start with the first four features and see how it goes*

### Years of Work Experience

In [32]:
a = datetime(2022, 10, 10)
b = datetime(2021, 2, 2)
a < b

False

In [33]:
# Possible values of time: {'', 'Aujourd’hui', 'Current', 'N/A', 'Present', 'Presente', 'Till Date', 'current', 'notKnown', 'present'}
unknown_time_values = set(['', 'notKnown', 'N/A', None])
today_time_values = set(['Aujourd’hui', 'o momento', 'Current', 'Present', 'Presente', 'Till Date', 'current', 'present'])


def merge_overlapping_intervals(array):
    array.sort() # sorts by 1st element of the 2-item list
    merged = [array[0]]
    for current in array:
        previous = merged[-1]
        if current[0] <= previous[1]:
            previous[1] = max(previous[1], current[1])
        else:
            merged.append(current)

    return merged


def extract_yoe(candidate: CandidateRecord):
    work_time_ranges = []
    for e in candidate.experiences:
        # If insufficient/invalid times are recorded for a given experience, we ignore them
        if len(e.time) < 2 or any([t in unknown_time_values for t in e.time]):
            continue
        start_time = dateparser.parse(e.time[0]) if e.time[0] not in today_time_values else dateparser.parse('today')
        end_time = dateparser.parse(e.time[1]) if e.time[1] not in today_time_values else dateparser.parse('today')
        work_time_ranges.append([start_time, end_time])
    

    if len(work_time_ranges) == 0:
        return 0
    else:
        work_time_ranges = merge_overlapping_intervals(work_time_ranges)
        yoe_in_months = sum([math.ceil((w[1] - w[0]).days / 30) for w in work_time_ranges]) 
        return yoe_in_months



### Creating feature vectors for train and test sets

In [34]:
# For each candidate, we will extract the features and then create a DataFrame from the records
# Length of feature vector = 1 (yoe) + 50 (degree) + 100 (title) + 200 (skills_uni) + 100 (skills_bitri) = 451
from tqdm import tqdm

def extract_features(c):
    c_vector = []

    c_vector.append(extract_yoe(c))

    degrees = [e.degree for e in c.educations]
    degrees = [s.translate(str.maketrans('', '', string.punctuation)) for s in degrees]
    c_vector += sum(degree_vectorizer.transform(degrees)).A[0].tolist() if degrees else [0]*50
    titles = [e.title for e in c.experiences]
    c_vector += sum(title_vectorizer.transform(titles)).A[0].tolist() if titles else [0]*100

    c_skills = []
    for e in c.experiences:
        if e.skills:
            c_skills += e.skills
    c_vector += sum(skills_vectorizer_unigram.transform(c_skills)).A[0].tolist() if c_skills else [0]*200
    c_vector += sum(skills_vectorizer_bitri.transform(c_skills)).A[0].tolist() if c_skills else [0]*100

    return c_vector
    

In [35]:
def extract_llm_features(c):
    c_vector = []
    c_vector.append(extract_yoe(c))

    text = ""
    for e in c.educations:
        text +=  (e.school if e.school else "") + (e.degree if e.degree else "") + (e.description if e.description else "")

    c_vector += get_distilbert_embeddings_averaged(model, tokenizer, text).tolist()

    text = ""
    for e in c.experiences:
        text += (e.employer if e.employer else "") + (e.title if e.title else "") + (e.description if e.description else "") + (' '.join(e.skills) if e.skills else "")

    c_vector += get_distilbert_embeddings_averaged(model, tokenizer, text).tolist()

    return c_vector
    


In [36]:
train_embeddings = []
for idx, c in enumerate(tqdm(train_candidates)):
    try:
        train_embeddings.append(extract_llm_features(c))
    except Exception as e:
        print("Encountered error while parsing train candidate:", idx)
        print(e)

test_embeddings = []
for idx, c in enumerate(tqdm(test_candidates)):
    try:
        test_embeddings.append(extract_llm_features(c))
    except Exception as e:
        print("Encountered error while parsing test candidate:", idx)
        print(e)

 76%|███████▋  | 22920/29999 [5:56:10<05:20, 22.10it/s]        

In [ ]:
import pickle
with open("./data/train_embeddings.pkl", 'wb') as out:
    pickle.dump(train_embeddings, out)
with open("./data/test_embeddings.pkl", 'wb') as out:
    pickle.dump(test_embeddings, out)

In [ ]:
train_df = []
for idx, c in enumerate(tqdm(train_candidates)):
    try:
        train_df.append(extract_features(c))
    except Exception as e:
        print("Encountered error while parsing train candidate:", idx)
        print(e)

test_df = []
for idx, c in enumerate(tqdm(test_candidates)):
    try:
        test_df.append(extract_features(c))
    except Exception as e:
        print("Encountered error while parsing test candidate:", idx)
        print(e)

100%|██████████| 10/10 [00:00<00:00, 9749.66it/s]


Encountered error while parsing train candidate: 0
name 'extract_features' is not defined
Encountered error while parsing train candidate: 1
name 'extract_features' is not defined
Encountered error while parsing train candidate: 2
name 'extract_features' is not defined
Encountered error while parsing train candidate: 3
name 'extract_features' is not defined
Encountered error while parsing train candidate: 4
name 'extract_features' is not defined
Encountered error while parsing train candidate: 5
name 'extract_features' is not defined
Encountered error while parsing train candidate: 6
name 'extract_features' is not defined
Encountered error while parsing train candidate: 7
name 'extract_features' is not defined
Encountered error while parsing train candidate: 8
name 'extract_features' is not defined
Encountered error while parsing train candidate: 9
name 'extract_features' is not defined


100%|██████████| 10/10 [00:00<?, ?it/s]

Encountered error while parsing test candidate: 0
name 'extract_features' is not defined
Encountered error while parsing test candidate: 1
name 'extract_features' is not defined
Encountered error while parsing test candidate: 2
name 'extract_features' is not defined
Encountered error while parsing test candidate: 3
name 'extract_features' is not defined
Encountered error while parsing test candidate: 4
name 'extract_features' is not defined
Encountered error while parsing test candidate: 5
name 'extract_features' is not defined
Encountered error while parsing test candidate: 6
name 'extract_features' is not defined
Encountered error while parsing test candidate: 7
name 'extract_features' is not defined
Encountered error while parsing test candidate: 8
name 'extract_features' is not defined
Encountered error while parsing test candidate: 9
name 'extract_features' is not defined


In [ ]:
with open("./data/train_features.pkl", 'wb') as out:
    pickle.dump(train_df, out)
with open("./data/test_features.pkl", 'wb') as out:
    pickle.dump(test_df, out)

In [ ]:
len(train_df), len(train_df[0]), len(test_df), len(test_df[0])

(29969, 451, 29999, 451)

### Create train and test datasets

In [ ]:
label_map = {
    None: -1,
    "Intern": 0,
    "Entry": 1,
    "Mid-Level": 2,
    "Senior": 3,
    "Manager": 4,
    "Director": 5,
    "Vice President": 6,
    "CXO": 7
}

y_train_all = [label_map[c.seniority] for c in train_candidates]
y_test = [label_map[c.seniority] for c in test_candidates]

In [ ]:
bad_train_indices = [idx for idx, label in enumerate(y_train_all) if label == -1]
bad_test_indices = [idx for idx, label in enumerate(y_test) if label == -1]

len(bad_train_indices), len(bad_test_indices)

(294, 272)

In [ ]:
X_train_all = pd.DataFrame(train_df)
X_test = pd.DataFrame(test_df)

X_train_all = X_train_all.drop(index=bad_train_indices)
X_test = X_test.drop(index=bad_test_indices)


In [ ]:
y_train_all = [label for idx, label in enumerate(y_train_all) if idx not in bad_train_indices]
y_test = [label for idx, label in enumerate(y_test) if idx not in bad_test_indices]

In [ ]:

X_train_all.shape, X_test.shape, len(y_train_all), len(y_test)

((29675, 451), (29727, 451), 29675, 29727)

# Model Training

I did not do comprehensive hyper-parameter tuning with a validation set, but I did try `n_estimators = [100, 200, 300]`, `max_depth = [5, 10, 15]`, and `learning_rate = [0.001, 0.01, 0.02, 0.03]`

I found that (300, 15, 0.03) fetched the minimum training error and maximum training accuracy (~97%).

In [56]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X_train_all, y_train_all, test_size=0.2, random_state=42)

x_train.shape, x_val.shape, len(y_train), len(y_val)

((23740, 451), (5935, 451), 23740, 5935)

In [60]:
clf = XGBClassifier(n_estimators=350, max_depth=10, learning_rate=0.03, n_jobs=-1, gpu_id=0, tree_method="gpu_hist")
clf.fit(x_train, y_train, eval_set=[(x_val, y_val)])

[0]	validation_0-mlogloss:2.02003
[1]	validation_0-mlogloss:1.96600
[2]	validation_0-mlogloss:1.91628
[3]	validation_0-mlogloss:1.87023
[4]	validation_0-mlogloss:1.82737
[5]	validation_0-mlogloss:1.78756
[6]	validation_0-mlogloss:1.74997
[7]	validation_0-mlogloss:1.71464
[8]	validation_0-mlogloss:1.68091
[9]	validation_0-mlogloss:1.64904
[10]	validation_0-mlogloss:1.61875
[11]	validation_0-mlogloss:1.59007
[12]	validation_0-mlogloss:1.56272
[13]	validation_0-mlogloss:1.53685
[14]	validation_0-mlogloss:1.51187
[15]	validation_0-mlogloss:1.48811
[16]	validation_0-mlogloss:1.46515
[17]	validation_0-mlogloss:1.44326
[18]	validation_0-mlogloss:1.42221
[19]	validation_0-mlogloss:1.40211
[20]	validation_0-mlogloss:1.38304
[21]	validation_0-mlogloss:1.36422
[22]	validation_0-mlogloss:1.34615
[23]	validation_0-mlogloss:1.32888
[24]	validation_0-mlogloss:1.31215
[25]	validation_0-mlogloss:1.29607
[26]	validation_0-mlogloss:1.28062
[27]	validation_0-mlogloss:1.26560
[28]	validation_0-mlogloss:1.2

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=350, n_jobs=-1, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

# Evaluation

In [61]:
train_preds = clf.predict(X_train_all)
accuracy_score(y_train_all, train_preds), balanced_accuracy_score(y_train_all, train_preds)

(0.8945240101095198, 0.8631036584015404)

In [62]:
preds = clf.predict(X_test)
accuracy_score(y_test, preds), balanced_accuracy_score(y_test, preds)

(0.6566421098664513, 0.4884903192200657)

In [63]:
confusion_matrix(y_test, preds)

array([[ 238,  519,   68,   38,   26,    5,    0,    0],
       [ 165, 6773,  974,  212,  193,   40,    5,    0],
       [  17,  550, 6005, 1033,  886,  538,   51,   10],
       [  10,  156, 1010, 2387,  360,  243,   28,    2],
       [   5,  120,  895,  236, 2435,  366,   16,    0],
       [   4,   40,  310,   58,  170, 1483,   58,   12],
       [   0,   12,  140,   43,   66,  209,  191,   13],
       [   2,    8,  105,   33,   22,   98,   27,    8]], dtype=int64)

Overall, clearly the model is overfitting and is unable to adapt to the test data. Nevertheless, this is a good start.

- We can add features related to work experience descriptions to see if it improves the performance.
- Another possible area of improvement is looking at the skills/titles/degrees and how they differ between train and test data. Since we are using CountVectorizer, it is quite possible that the top n-grams in train and test are different.